In [10]:
from matplotlib import font_manager, rc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings(action='ignore')

sns.set(font_scale=1.4)
# 한글 폰트 사용을 위해서 세팅
font_path = "C:/Windows/Fonts/malgun.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

pd.set_option('display.max_columns', 500)


In [44]:
# 클러스터 개수, 데이터 버전
ver = 1
n_clusters = 9

# 데이터 가져오기
x = pd.read_csv('./data/data_clustering_sobi_ver{}.csv'.format(ver))
og = pd.read_csv('./data/data_2019_2021_revised.csv').sort_values(by='조사연월')
# 클러스터 라벨
pred = pd.read_csv('clustering_results/pred_HC_e50_k9_v1.csv')

x['cluster'] = pred
x['unique_id'] = og['가구일련번호']


In [45]:
x

,식료품비주류음료구입비,주류담배구입비,의류신발구입비,주거수도광열비,가정용품가사서비스이용금액,보건제품구입비,교통비,통신비,오락문화비,교육비,음식숙박비,기타상품서비스이용금액,cluster,unique_id
0,0.354795,0.004935,0.015852,0.202862,0.003963,0.023131,0.047776,0.073053,0.010059,0.000000,0.091774,0.171799,1,1
1,0.187313,0.001025,0.010843,0.199045,0.009981,0.110230,0.076543,0.057171,0.022645,0.022028,0.123908,0.179266,1,2
2,0.125368,0.001801,0.097792,0.252296,0.035849,0.040166,0.018412,0.023273,0.287130,0.000000,0.106167,0.011747,2,3
3,0.135129,0.004398,0.037949,0.124869,0.001438,0.017652,0.123128,0.107355,0.041332,0.017592,0.289794,0.099364,5,4
4,0.033236,0.006272,0.068057,0.034982,0.554156,0.031323,0.004408,0.007514,0.090670,0.000000,0.025642,0.143741,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72225,0.177179,0.034906,0.015222,0.033806,0.018089,0.047683,0.121903,0.043027,0.028867,0.092615,0.297132,0.089570,5,6547
72226,0.145644,0.008737,0.066905,0.065736,0.022591,0.009949,0.020063,0.038557,0.087088,0.058650,0.311238,0.164843,5,6548
72227,0.085509,0.010416,0.082540,0.022762,0.036786,0.014289,0.173627,0.241265,0.115185,0.000000,0.163013,0.054607,5,6549
72228,0.187142,0.021166,0.026734,0.091492,0.078667,0.061743,0.174814,0.037922,0.026923,0.000000,0.146716,0.146682,5,6550


In [46]:
def transition_matrix(data):

    # 그룹 별 월별 membership 리스트 만들기

    membership_dict = {}

    for key, value in data.groupby(['unique_id']):
        membership_dict[key] = value['cluster'].values.tolist()

    # transition matrix 만들기 (9 min)

    transition_df = pd.DataFrame(columns=['state', 'next_state'])

    for key, value in tqdm(membership_dict.items()):
        for i, val in enumerate(value[:-1]):  # We don't care about last state
            df_stg = pd.DataFrame(index=[0])
            df_stg['state'], df_stg['next_state'] = value[i], value[i+1]
            transition_df = pd.concat([transition_df, df_stg], axis=0)

    # 예를들어 16개월동안 모두 값이 존재했으면 15 transitions
    print('total number of transitions: ', len(transition_df))

    cross_tab = pd.crosstab(
        transition_df['state'], transition_df['next_state'])
    transition_matrix = cross_tab.div(cross_tab.sum(axis=1), axis=0)

    return transition_matrix, membership_dict


def color_df(df):

    cm = sns.light_palette("orange", as_cmap=True)
    x = pd.DataFrame(df)
    x = x.style.background_gradient(cmap=cm, axis=1)
    display(x)


In [47]:
matrix, membership_dict = transition_matrix(x)


100%|██████████| 6558/6558 [00:58<00:00, 111.44it/s]

total number of transitions:  65672


In [48]:
color_df(matrix)


next_state,0,1,2,3,4,5,6,7,8
state,,,,,,,,,
0,0.170967,0.168068,0.099352,0.079733,0.073258,0.217938,0.131149,0.005605,0.053929
1,0.152442,0.171328,0.093624,0.078694,0.081662,0.214947,0.139671,0.006565,0.061067
2,0.163492,0.167778,0.095079,0.081587,0.070317,0.216667,0.142222,0.007937,0.054921
3,0.158288,0.165357,0.087785,0.089159,0.073252,0.212883,0.138452,0.006874,0.067950
4,0.145049,0.169709,0.094563,0.078447,0.079417,0.225825,0.139612,0.005243,0.062136
5,0.150669,0.167687,0.093349,0.076047,0.080675,0.224651,0.143762,0.005340,0.057818
6,0.144444,0.170697,0.093682,0.067320,0.076253,0.217102,0.162527,0.007407,0.060566
7,0.133495,0.152913,0.082524,0.070388,0.097087,0.206311,0.169903,0.012136,0.075243
8,0.148213,0.163853,0.091112,0.079196,0.080189,0.221946,0.140020,0.004965,0.070506


In [49]:
membership_dict

{1: [1, 6, 8, 8, 2, 3, 3, 3, 5, 5, 0, 5],
 2: [1, 1, 5, 4, 0, 5, 6, 6, 3, 2, 3, 4],
 3: [2, 8, 8, 0, 5, 3, 5, 5, 2, 3, 5, 1],
 4: [5, 5, 3, 5, 3, 3, 5, 5, 3, 1, 5, 5],
 5: [1, 1, 4, 5, 5, 5, 3, 5, 1, 3, 4, 0],
 6: [6, 6, 5, 2, 5, 5, 2, 5, 3, 5, 6, 6],
 7: [8, 8, 2, 1, 1, 6, 5, 1, 5, 4, 1, 5],
 8: [1, 0, 3, 2, 3, 0, 3, 6, 2, 4, 5, 5],
 9: [5, 5, 2, 2, 4, 5, 5, 3, 5, 0, 6, 4],
 10: [1, 5, 6, 5, 3, 1, 1, 1, 1, 5, 2, 2],
 11: [2, 8, 1, 8, 1, 1, 3, 1, 6, 1, 1, 0],
 12: [5, 5, 1, 1, 1, 2, 0, 2, 8, 1, 4, 0],
 13: [2, 5, 5, 5, 8, 5, 5, 5, 0, 6, 6, 1],
 14: [2, 4, 8, 5, 1, 8, 0, 6, 2, 1, 0, 3],
 15: [2, 2, 0, 8, 5, 2, 0, 5, 1, 8, 5, 3],
 16: [5, 2, 1, 2, 6, 6, 5, 1, 0, 5, 1, 6],
 17: [2, 5, 1, 3, 0, 1, 1, 6, 2, 5, 6, 6],
 18: [5, 2, 5, 6, 3, 0, 6, 0, 8, 6, 2, 6],
 19: [1, 5, 2, 2, 8, 5, 0, 5, 6, 6, 6, 5],
 20: [6, 5, 5, 4, 5, 8, 0, 0, 5, 6, 6, 5],
 21: [0, 6, 4, 2, 5, 5, 0, 5, 8, 5, 6, 5],
 22: [3, 2, 2, 2, 8, 2, 5, 5, 2, 5, 5, 0],
 23: [2, 2, 6, 5, 2, 1, 1, 5, 5, 5, 5, 5],
 24: [8, 2, 6, 8, 2,

In [50]:
og

,조사연월,가구일련번호,가구구분코드,가구원수,취업인원수,노인가구여부,모자가구여부,맞벌이가구여부,일반가구여부,세대구분코드,배우자코드,가구주_관계코드,가구주_성별코드,가구주_연령,가구주_학력코드,가구주_수학구분코드,가구주_취업여부,가구주_10차산업분류코드,가구주_7차직업분류코드,가구주_종사상지위코드,가구원2_가구주관계코드,가구원2_성별코드,가구원2_연령,가구원2_학력코드,가구원2_수학구분코드,가구원2_취업여부,가구원2_10차산업분류코드,가구원2_7차직업분류코드,가구원2_종사상지위코드,가구원3_가구주관계코드,가구원3_성별코드,가구원3_연령,가구원3_학력코드,가구원3_수학구분코드,가구원3_취업여부,가구원3_10차산업분류코드,가구원3_7차직업분류코드,가구원3_종사상지위코드,가구원4_가구주관계코드,가구원4_성별코드,가구원4_연령,가구원4_학력코드,가구원4_수학구분코드,가구원4_취업여부,가구원4_10차산업분류코드,가구원4_7차직업분류코드,가구원4_종사상지위코드,가구원5_가구주관계코드,가구원5_성별코드,가구원5_연령,가구원5_학력코드,가구원5_수학구분코드,가구원5_취업여부,가구원5_10차산업분류코드,가구원5_7차직업분류코드,가구원5_종사상지위코드,가구원6_가구주관계코드,가구원6_성별코드,가구원6_연령,가구원6_학력코드,가구원6_수학구분코드,가구원6_취업여부,가구원6_10차산업분류코드,가구원6_7차직업분류코드,가구원6_종사상지위코드,가구원7_가구주관계코드,가구원7_성별코드,가구원7_연령,가구원7_학력코드,가구원7_수학구분코드,가구원7_취업여부,가구원7_10차산업분류코드,가구원7_7차직업분류코드,가구원7_종사상지위코드,가구원8_가구주관계코드,가구원8_성별코드,가구원8_연령,가구원8_학력코드,가구원8_수학구분코드,가구원8_취업여부,가구원8_10차산업분류코드,가구원8_7차직업분류코드,가구원8_종사상지위코드,가구원9_가구주관계코드,가구원9_성별코드,가구원9_연령,가구원9_학력코드,가구원9_수학구분코드,가구원9_취업여부,가구원9_10차산업분류코드,가구원9_7차직업분류코드,가구원9_종사상지위코드,비동거취업배우자유무,비동거학업배우자유무,비동거기타배우자유무,비동거취업미혼자녀수,비동거학업미혼자녀수,비동거기타미혼자녀수,거처구분코드,자동차보유대수,입주형태코드,월세평가금액,전세보증금,월세사글세,주거용전용면적,주택소유유무,가중값,전국전체가구_1인이상소득5분위코드,전국근로자가구_1인이상소득5분위코드,전국근로자외가구_1인이상소득5분위코드,도시전체가구_1인이상소득5분위코드,도시근로자가구_1인이상소득5분위코드,도시근로자외가구_1인이상소득5분위코드,전국전체가구_1인이상소득10분위코드,전국근로자가구_1인이상소득10분위코드,전국근로자외가구_1인이상소득10분위코드,도시전체가구_1인이상소득10분위코드,도시근로자가구_1인이상소득10분위코드,도시근로자외가구_1인이상소득10분위코드,소득구간코드,소득,소득_경상소득,소득_경상소득_근로소득,소득_경상소득_근로소득_가구주근로소득,소득_경상소득_근로소득_배우자근로소득,소득_경상소득_근로소득_기타가구원근로소득,소득_경상소득_사업소득,소득_경상소득_사업소득_가구주사업소득,소득_경상소득_사업소득_배우자사업소득,소득_경상소득_사업소득_기타가구원사업소득,소득_경상소득_사업소득_주택등임대소득,소득_경상소득_재산소득,소득_경상소득_재산소득_이자소득,소득_경상소득_재산소득_배당소득,소득_경상소득_재산소득_개인연금소득,소득_경상소득_재산소득_퇴직연금소득,소득_경상소득_재산소득_기타재산소득,소득_경상소득_이전소득,소득_경상소득_이전소득_공적연금,소득_경상소득_이전소득_기초연금,소득_경상소득_이전소득_사회수혜금액,소득_경상소득_이전소득_사회적현물이전금액,소득_경상소득_이전소득_연말정산환급금,소득_경상소득_이전소득_가구간이전소득,소득_경상소득_이전소득_할인혜택금액,소득_경상소득_이전소득_기타이전소득,소득_비경상소득,소득_비경상소득_경조소득,소득_비경상소득_퇴직수당,소득_비경상소득_기타비경상소득,가계지출금액,가계지출_소비지출금액,가계지출_비소비지출금액,도시여부,식료품비주류음료구입비,주류담배구입비,의류신발구입비,주거수도광열비,가정용품가사서비스이용금액,보건제품구입비,교통비,통신비,오락문화비,교육비,음식숙박비,기타상품서비스이용금액,_경상조세,_비경상조세,_연금기여금,_사회보험료,_이자비용,_가구간이전지출금액,_비영리단체이전지출금액
0,201914,1,2,1,0,0,0,0,1,1,3,1,2,64,6,1,2,Z,Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1,0,4,400,40000,0,50.0,2,6267.7609,1,NaN,1.0,1.0,NaN,1.0,1,NaN,1.0,1.0,NaN,1.0,1,425000,425000,150000,150000,0,0,0,0,0,0,0,0,0,0,0,0,0,275000,0,0,1933,0,6400,266667,0,0,0,0,0,0,640295,462581,177713,1,164121,2283,7333,93840,1833,10700,22100,33793,4653,0,42453,79471,0,30380,0,0,0,100000,47333
2959,201914,2960,2,2,1,0,0,0,1,1,1,1,1,58,6,1,2,Z,Z,NaN,2.0,2.0,57.0,6.0,1.0,1.0,P,2,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,3,2,1,400,0,0,115.0,1,1044.7004,3,NaN,4.0,3.0,NaN,4.0,6,NaN,8.0,6.0,NaN,7.0,4,3879300,3879300,0,0,0,0,3879300,1879300,2000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2470520,1366520,1104000,1,180010,0,0,178300,14200,286500,279300,165968,16242,0,134000,112000,0,32000,295000,237000,0,500000,40000
2960,201914,2961,1,2,1,0,0,0,1,2,3,1,1,46,4,1,1,H,9,1.0,3.0,2.0,7.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,2,1,5,450,27819,91,16.0,2,3175.5313,3,2.0,NaN,3.0,2.0,NaN,5,4.0,NaN,5.0,4.0,NaN,4,3363364,3363364,3150000,3150000,0,0,0,0,0,0,0,0,0,0,0,0,0,213364,0,0,0,180031,0,33333,0,0,0,0,0,0,2965462,2476095,489367,1,280908,60240,55727,458727,42500,47067,162567,502